In [ ]:
import sys
sys.path.append("..")
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec

from scipy import fft


import lauetools.LaueTools.IOimagefile as IOimage

#from libtiff import TIFF

from sklearn import cluster
from sklearn.feature_extraction.image import grid_to_graph

from sklearn import metrics


nx,ny=64,146

#X=np.load('spatial_X.npy')#[:nx,60:60+ny,:]
#argmin=X[:,0].argmin()
FA_spatial=np.load('full_spatial_features.npy')
#RS_spatial = agglo.inverse_transform(FA_spatial)
#RS_spatial=RS_spatial.reshape((nx,ny,128,128))



from sklearn.decomposition import PCA


sil=dict()
sil[512]=[]
sil[1024]=[]
sil[2048]=[]
sil[4096]=[]

cal=dict()
cal[512]=[]
cal[1024]=[]
cal[2048]=[]
cal[4096]=[]

dav=dict()
dav[512]=[]
dav[1024]=[]
dav[2048]=[]
dav[4096]=[]

ns_clusters_=np.arange(50,950,25)

for n_pca in (512,1024,2048,4096):
    pca = PCA(n_components=n_pca, svd_solver='randomized')
    pca_features=pca.fit_transform(FA_spatial)#components_
    connectivity = grid_to_graph(n_x=nx, n_y=ny)
    for n_clusters_ in ns_clusters_:
        ward = cluster.AgglomerativeClustering(n_clusters=n_clusters_, connectivity=connectivity, linkage='ward').fit(pca_features)
        labels = ward.labels_
        #sample_silhouette_values = metrics.silhouette_samples(pca_features, labels)
        sil[n_pca].append(metrics.silhouette_score(pca_features,labels))
        cal[n_pca].append(metrics.calinski_harabasz_score(pca_features,labels))
        dav[n_pca].append(metrics.davies_bouldin_score(pca_features,labels))

plt.plot(ns_clusters_,sil[n_pca],'k^--')
plt.plot(ns_clusters_,cal[n_pca],'ko--')
plt.plot(ns_clusters_,dav[n_pca],'k+--')
